In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Read in the all the City Data
City_Temp = "Resources/city_temperature.csv"
City_Temp_df = pd.read_csv(City_Temp)
City_Temp_df.head()

C:\Users\AjgOI\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [3]:
# Read in the all the City Temperature Data
Env_Temp = "Resources/Environment_Temperature_change.csv"
Env_Temp_df = pd.read_csv(Env_Temp, encoding='ISO-8859-1')
Env_Temp_df

,Area Code,Area,Months Code,Months,Element Code,Element,Y1961,Y1962,Y1963,Y1964,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,0.777,0.062,2.744,-5.232,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,1.950,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,-1.743,2.465,3.919,-0.202,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,2.597,2.597,2.597,2.597,...,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,0.516,1.336,0.403,1.659,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,OECD,7018,JunJulAug,6078,Standard Deviation,0.247,0.247,0.247,0.247,...,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247
9652,5873,OECD,7019,SepOctNov,7271,Temperature change,0.036,0.461,0.665,-0.157,...,0.958,1.106,0.885,1.041,0.999,1.670,1.535,1.194,0.581,1.233
9653,5873,OECD,7019,SepOctNov,6078,Standard Deviation,0.378,0.378,0.378,0.378,...,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378
9654,5873,OECD,7020,Meteorological year,7271,Temperature change,0.165,-0.009,0.134,-0.190,...,1.246,0.805,1.274,0.991,0.811,1.282,1.850,1.349,1.088,1.297


In [4]:
#Filter the EnviroTemp Dataframe

Area = Env_Temp_df.loc[:,["Area Code","Area"]]
Month = Env_Temp_df.loc[:,["Months Code","Months"]]
Element = Env_Temp_df.loc[:,["Element Code","Element"]]

#Remove Duplicates from all Data frames
Area.drop_duplicates("Area Code", inplace=True)
Area.set_index("Area Code", inplace=True)

Month.drop_duplicates("Months Code", inplace=True)
Month.set_index("Months Code", inplace=True)

Element.drop_duplicates("Element Code", inplace=True)
Element.set_index("Element Code", inplace=True)


In [5]:
engine = create_engine('postgresql://postgres:@Jg211083@localhost:5432/temperaturedb')
connection = engine.connect()

In [6]:
#Create three new tables with Supplimentary information relatin back to the original data table.

# Create Areas table
Area.reset_index()
Area.to_sql(name='EnvArea',con=connection, if_exists='replace', index=True)

# Create Elements Table
Element.reset_index()
Element.to_sql(name='Element',con=connection, if_exists='replace', index=True)

#Create a Month table
Month.reset_index()
Month.to_sql(name='Month', con=connection, if_exists='replace', index=True)



In [7]:
#Create two seperate filtered tables for Temperature Variance and Standard Dev data tables

Temperature_Var = Env_Temp_df.loc[(Env_Temp_df["Element"] == "Temperature change")]

Std_Dev = Env_Temp_df.loc[(Env_Temp_df["Element"]== "Standard Deviation")]

Temperature_Var.to_sql(name='TempVar', con=connection, if_exists='replace', index=True)

Std_Dev.to_sql(name='Std_Dev', con=connection, if_exists='replace', index=True)


In [19]:
# Cities List imported in to a unique table

Cities = City_Temp_df.loc[:,"City"]

Cities.drop_duplicates()

Cities_Unique = pd.DataFrame(Cities.unique())

Cities_Unique.columns = ['City']

Cities_Unique.to_sql(name='Cities', con=connection, if_exists='replace', index=True)

In [20]:
# Create a unique Countries List

Countries = City_Temp_df.loc[:,"Country"]

Countries.drop_duplicates()

Counties_Unique = pd.DataFrame(Countries.unique())

Counties_Unique.columns = ['Country']

Counties_Unique.to_sql(name='Countries', con=connection, if_exists='replace', index=True)

In [21]:
# Create a unique list of regions

Regions = City_Temp_df.loc[:,"Region"]

Regions.drop_duplicates()

Regions_Unique = pd.DataFrame(Regions.unique())

Regions_Unique.columns = ["Region"]

Regions_Unique.to_sql(name='Regions', con=connection, if_exists='replace', index=True, )

In [22]:
# Removal of State field

Cities_Final = City_Temp_df.drop(['State'], axis=1)
Cities_Final.to_sql(name='CityTemp', con=connection, if_exists='replace', index=True)

In [35]:
# Dependencies
import matplotlib.pyplot as plt
import requests
from config import weather_api_key

In [36]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [45]:
City_list = []
City_list = Cities_Unique.City.to_list()

# set up lists to hold reponse info
lat = []
temp = []
exceptions = []
Current_City = []
# Loop through the list of cities and perform a request for data on each
for city in City_list:
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        Current_City.append(city)
    except:
        print(f'No data found for city: {city}')
        exceptions.append(city)
        pass


print(f"The latitude information received is: {lat}")
print(f"The temperature information received is: {temp}")

No data found for city:Capetown
No data found for city:Bombay (Mumbai)
No data found for city:Chennai (Madras)
No data found for city:Ulan-bator
No data found for city:Dusanbe
No data found for city:Ashabad
No data found for city:Port au Prince
No data found for city:Montvideo
No data found for city:Hartford Springfield
No data found for city:Tampa St. Petersburg
No data found for city:Sault Ste Marie
No data found for city:Minneapolis St. Paul
No data found for city:Raleigh Durham
No data found for city:Akron Canton
No data found for city:Wilkes Barre
No data found for city:Dallas Ft Worth
No data found for city:Midland Odessa
No data found for city:San Juan Puerto Rico
The latitude information received is: [36.7525, -3.3822, 6.3654, 4.3612, -4.2658, 30.0626, 9.025, 0.3925, 13.45, 9.538, 11.8636, 5.3094, -1.2833, 33.9911, -18.9137, 18.0858, -13.9669, -25.9653, -22.5594, 13.5137, 6.5833, 14.6937, 8.484, 6.1375, 34, -6.8235, 0.3163, -15.4067, 23.7104, 39.9075, 30.6667, 23.1167, 31.2222,

In [48]:
# create a data frame from cities, lat, and temp
weather_dict = {
    "city": Current_City,
    "lat": lat,
    "temp": temp
}

weather_data = pd.DataFrame(weather_dict)
weather_data.head()

# Create a new SQL data table with the Current City Data
weather_data.to_sql(name='CurrentCityWeather', con=connection, if_exists='replace', index=True)